# Optmising Parameters

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hyperparameters

We define the following hyperparameters for training:
- **Number of Epochs** - the number times to iterate over the dataset
- **Batch Size** - the number of data samples propagated through the network before the parameters are updated
- **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

### Optimization Loop
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
- **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
- **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

### Loss Function
- Common cost functions include `nn.MSELoss` for regression tasks.
- `nn.NLLLoss` for classification.
- `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

We pass our model’s output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error.

In [2]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimiser

Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed.

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In training there are three steps to optimisation
- Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

- Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.

- Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

# Full Code

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312244  [   64/60000]
loss: 2.288803  [ 6464/60000]
loss: 2.268211  [12864/60000]
loss: 2.259167  [19264/60000]
loss: 2.244292  [25664/60000]
loss: 2.218633  [32064/60000]
loss: 2.224800  [38464/60000]
loss: 2.186306  [44864/60000]
loss: 2.182074  [51264/60000]
loss: 2.147323  [57664/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 2.139834 

Epoch 2
-------------------------------
loss: 2.155960  [   64/60000]
loss: 2.141011  [ 6464/60000]
loss: 2.080541  [12864/60000]
loss: 2.095137  [19264/60000]
loss: 2.045035  [25664/60000]
loss: 1.986126  [32064/60000]
loss: 2.011204  [38464/60000]
loss: 1.923838  [44864/60000]
loss: 1.930889  [51264/60000]
loss: 1.858214  [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.850399 

Epoch 3
-------------------------------
loss: 1.890239  [   64/60000]
loss: 1.857987  [ 6464/60000]
loss: 1.733669  [12864/60000]
loss: 1.773687  [19264/60000]
loss: 1.670036  [25664/60000]
loss: 1.628949  [32064/600